In [1]:
from _Setup import *

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif

In [2]:
train = pd.read_csv(missForestTrainFile)
test = pd.read_csv(missForestTestFile)
submission_data = pd.read_csv(rawSubmissionFile)

In [3]:
#Separating columns 
X = train.drop(columns=['Unit2','Unnamed: 0', 'HospAdmTime','SepsisLabel', 'time'])  
y = train['SepsisLabel']

In [4]:
X.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,pH,...,Potassium,Hct,Hgb,WBC,Platelets,Age,patient,Gender,Unit1,ICULOS
0,-0.432220,0.749512,-2.124421,-0.621283,0.146051,0.420292,-1.294263,0.045536,0.073114,0.340467,...,0.868277,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,1.0
1,-0.432220,0.749512,-2.124421,-0.621283,0.146051,0.420292,-1.294263,0.045536,0.073114,0.340467,...,0.868277,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,2.0
2,-0.320708,0.592277,-0.914129,-0.428503,0.299849,0.716228,-1.294263,0.045536,-0.000054,0.710267,...,0.868277,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,3.0
3,-0.376464,0.906747,-0.531932,-0.471343,0.053772,0.346308,-1.294263,-0.768098,-0.000054,-0.029333,...,0.868277,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,4.0
4,-0.710999,0.906747,-0.531932,-0.342823,-0.130785,0.050372,-1.479445,-0.768098,-0.014687,-0.029333,...,0.868277,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,5.0


In [5]:
#k = 10  # Adjust this number based on your needs
#selector = SelectKBest(score_func=f_classif, k=k)
#X_selected = selector.fit_transform(X, y)

# Get the names of the selected features
#selected_features = X.columns[selector.get_support()]
#print("Selected Features:", selected_features)

In [6]:
# selecting variables 
#selected_columns = ['ICULOS', 'Temp', 'Resp', 'BaseExcess']
selected_columns = ['ICULOS']
X_selected = X[selected_columns]

In [ ]:
# take a sample from the data 

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear') 
model.fit(X_selected, y)

In [ ]:
y_pred = model.predict(test)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
}

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")

In [ ]:
submission_data['SepsisLabel'] = y_pred_test

# Optionally, save the modified DataFrame to a CSV file
# TODO : create a variable for this path in _Setup.py
# submission_data.to_csv('submission_data_with_predictions.csv', index=False)

# Display the updated DataFrame (optional)
print(submission_data.head())

In [ ]:
num_rows = submission_data.shape[0]
print("Number of rows in submission_data:", num_rows)

In [ ]:
print("\nClassification Report (Training):\n", classification_report(y_train, y_pred_train))